In [1]:
import torch
import numpy as np
import pgnn_control as pgnnc
from robot_env import robot_env
from utils import to_tensors, combine_state, wrap_to_pi, rotate
import os
# cwd = os.path.dirname(os.path.realpath(__file__))
from print_xacros import get_names
from apply_policy import apply_policy, make_goal_memory
from planning_utils import compare_velocities

all_name_list = get_names()
print(len(all_name_list), ' designs')


# divide them up into seen and unseen
unseen_inds = []
seen_inds = []
for i_urdf in range(len(all_name_list)):
    urdf = all_name_list[i_urdf]
    if (urdf[0:3]==urdf[3:6][::-1]):
        seen_inds.append(i_urdf)
    else:
        unseen_inds.append(i_urdf)
print(len(seen_inds), ' seen designs')
print(len(unseen_inds), ' unseen designs')


144  designs
12  seen designs
132  unseen designs


In [2]:
device = torch.device('cpu')


# make some direction goals
goal_memory = make_goal_memory(41) # 10*4 + 1

# load up a learned policy to test
# # load previous weights if desired
folder = 'mbrl_v4_test15B'
control_fname = os.path.join(folder, 'multidesign_control_iter3.pt')
if os.path.exists(control_fname):
    print('Loading weights from ' + control_fname)
    save_dict = torch.load( control_fname, map_location=lambda storage, loc: storage)

    gnn_nodes_control = pgnnc.create_GNN_nodes(save_dict['internal_state_len'], 
                save_dict['message_len'], save_dict['hidden_layer_size'], 
                device, goal_len=3, body_input= True) 

    pgnnc.load_state_dicts(gnn_nodes_control, save_dict['gnn_state_dict'])

data_subfolder = os.path.join(folder,'zero_shot_data')
if not(os.path.exists(data_subfolder)):
    os.mkdir(data_subfolder)
    print('Created folder ' + data_subfolder)
else:
    print('Using folder ' + data_subfolder)
    

Loading weights from mbrl_v8_test9/multidesign_control_iter3.pt
Created folder mbrl_v8_test9/zero_shot_data


In [3]:
urdf_names = all_name_list
# random_choice = np.random.choice(len(unseen_inds), 8*6, replace=False)
# # urdf_names = [all_name_list[unseen_inds[ind]] for ind in random_choice]
# print(urdf_names)
urdf_names = ['llllll', 'wnwwnw', 'llwwll', 'lnwwnl']
urdf_names += [ 'lnllnl', 'lwllwl', 'lwwwwl', 'wlwwlw', 
               'wwllww', 'wwwwww', 'wnllnw', 'wllllw']

CREATE_VIDEOS= False
# CREATE_VIDEOS= True


T = 20
vel_metric_list = []
vel_baseline_list = []
video_names = []
for i_urdf in range(len(urdf_names)):
    urdf = urdf_names[i_urdf]

    save_path = os.path.join(data_subfolder, urdf + '_apply_policy.ptx')
    video_name = os.path.join(data_subfolder, urdf + '_goal')
    video_names.append(video_name)
    if CREATE_VIDEOS:
        apply_policy(urdf, goal_memory,
          gnn_nodes_control, device, save_path, show_GUI=CREATE_VIDEOS, 
                 video_name = video_name, sim_speed_factor=10)
    else:
        apply_policy(urdf, goal_memory, 
                     gnn_nodes_control, device, save_path, show_GUI=False)

    save_dict = torch.load(save_path, map_location=lambda storage, loc: storage)
    vel_metric, vel_metric_baseline = compare_velocities(
            save_dict['states_memory'],
            save_dict['goal_memory'], 
            save_dict['run_lens'],
            10, T )
    vel_metric_list.append(vel_metric)
    vel_baseline_list.append(vel_metric_baseline)
    print(urdf + ': ' + str(np.round(vel_metric,2))
          + ' baseline ' + str(np.round(vel_metric_baseline,2))
          + ' ' + str(i_urdf) + '/' + str(len(urdf_names)))

llllll: 0.16 baseline 0.23 0/12
wnwwnw: 0.08 baseline 0.23 1/12
llwwll: 0.13 baseline 0.23 2/12
lnwwnl: 0.16 baseline 0.23 3/12
lnllnl: 0.15 baseline 0.23 4/12
lwllwl: 0.1 baseline 0.23 5/12
lwwwwl: 0.1 baseline 0.23 6/12
wlwwlw: 0.08 baseline 0.23 7/12
wwllww: 0.09 baseline 0.23 8/12
wwwwww: 0.09 baseline 0.23 9/12
wnllnw: 0.13 baseline 0.23 10/12
wllllw: 0.12 baseline 0.23 11/12


In [4]:

if len(urdf_names)>50:
    vel_data_path = os.path.join(data_subfolder, 'transfer_results.ptx')
    vel_dict = dict()
    vel_dict['unseen_inds'] = unseen_inds
    vel_dict['seen_inds'] = seen_inds
    vel_dict['urdf_names'] = urdf_names
    vel_dict['vel_metric_list'] = vel_metric_list
    vel_dict['vel_baseline_list'] = vel_baseline_list
    torch.save(vel_dict, vel_data_path)

    vel_save_path = os.path.join(data_subfolder, 'transfer_results.csv')
    seen_names = [urdf_names[s] for s in seen_inds]
    unseen_names = [urdf_names[s] for s in unseen_inds]

    vel_metric_list = np.array(vel_metric_list)
    vel_baseline_list = np.array(vel_baseline_list)

    with open(vel_save_path, 'w') as fp:
        names_text = ''
        for urdf in unseen_names:
            names_text = names_text + urdf + ',' 

        fp.write('--- Unseen Names: ---\n')
        fp.write(names_text + '\n')
        fp.write('Metric Mean: ' + str(np.mean(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Min: ' + str(np.min(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Max: ' + str(np.max(vel_metric_list[unseen_inds]))+'\n')
        fp.write('Metric Rescaled: ' + str(
            np.mean( (vel_baseline_list[unseen_inds] - vel_metric_list[unseen_inds])
                    /vel_baseline_list[unseen_inds])
            )+'\n')

        fp.write('--- Metric: ---\n')
        np.savetxt(fp, vel_metric_list[unseen_inds], delimiter=',')   
        fp.write('--- Baseline: ---\n')
        np.savetxt(fp, vel_baseline_list[unseen_inds], delimiter=',')

        names_text = ''
        for urdf in seen_names:
            names_text = names_text + urdf + ',' 

        fp.write('--- Seen Names: ---\n')
        fp.write(names_text + '\n')
        fp.write('Metric Mean: ' + str(np.mean(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Min: ' + str(np.min(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Max: ' + str(np.max(vel_metric_list[seen_inds]))+'\n')
        fp.write('Metric Rescaled: ' + str(
            np.mean( (vel_baseline_list[seen_inds] - vel_metric_list[seen_inds])
                    /vel_baseline_list[seen_inds])
            )+'\n')
        fp.write('--- Metric: ---\n')
        np.savetxt(fp, vel_metric_list[seen_inds], delimiter=',')   
        fp.write('--- Baseline: ---\n')
        np.savetxt(fp, vel_baseline_list[seen_inds], delimiter=',')
        print('wrote file')

In [5]:
video_names

['mbrl_v4_test15B/zero_shot_data/llllll_goal',
 'mbrl_v4_test15B/zero_shot_data/wnwwnw_goal',
 'mbrl_v4_test15B/zero_shot_data/llwwll_goal',
 'mbrl_v4_test15B/zero_shot_data/lnwwnl_goal',
 'mbrl_v4_test15B/zero_shot_data/lnllnl_goal',
 'mbrl_v4_test15B/zero_shot_data/lwllwl_goal',
 'mbrl_v4_test15B/zero_shot_data/lwwwwl_goal',
 'mbrl_v4_test15B/zero_shot_data/wlwwlw_goal',
 'mbrl_v4_test15B/zero_shot_data/wwllww_goal',
 'mbrl_v4_test15B/zero_shot_data/wwwwww_goal',
 'mbrl_v4_test15B/zero_shot_data/wnllnw_goal',
 'mbrl_v4_test15B/zero_shot_data/wllllw_goal']

In [6]:
# # # video tile compile. This works ok
import subprocess

for i in range(6):
    end_str = str(i) + '.mp4' # add the goal number and filetype to the video names
    
    # get the number of frames from the video segments
    print('getting video lengths')
    vid_frames = []
    for vid in video_names:
        batcmd = ('ffprobe -select_streams v -show_streams '
               + vid+end_str)
        result = str(subprocess.check_output(batcmd, shell=True))
        nb_frames_index = result.find('nb_frames=')
        next_backslash = result[nb_frames_index:].find('\\n')
        n_frames = result[nb_frames_index+len('nb_frames='):
                          nb_frames_index+next_backslash]
        vid_frames.append(int(n_frames))

    # calculate retiming so that all segments can be retimed to be the same length
    print('relative retiming: ' + str(np.array(vid_frames)/np.max(vid_frames)))
    retiming = 3*np.max(vid_frames)/np.array(vid_frames) # slow down by 3x (about realtime)
    
    # tile videos side by side (2x2)
#     run_str = ('ffmpeg'+
#                ' -i '+ video_names[0]+end_str+
#                ' -i '+ video_names[1]+end_str+ 
#                ' -i '+ video_names[2]+end_str+
#                ' -i '+ video_names[3]+end_str+
#                ' -filter_complex '+ 
#                '"nullsrc=size=2048x1536 [base]; '+
#                '[0:v] setpts=' + str(retiming[0]) +'*PTS, scale=1024x768 [upperleft]; '+
#                '[1:v] setpts=' + str(retiming[1]) +'*PTS, scale=1024x768 [upperright]; '+
#                '[2:v] setpts=' + str(retiming[2]) +'*PTS, scale=1024x768 [lowerleft]; '+
#                '[3:v] setpts=' + str(retiming[3]) +'*PTS, scale=1024x768 [lowerright]; '+
#                '[base][upperleft] overlay=shortest=1 [tmp1]; '+
#                '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
#                '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
#                '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
#                os.path.join(data_subfolder,'output'+end_str)
#               )
    
    # tile videos side by side (4x3)
    run_str = 'ffmpeg'
    nv = len(video_names)
    
    # many videos
    n_rows = 3*2
    n_cols = 4*2
    
    # 12 videos
    n_rows = 3
    n_cols = 4
    
    for ii in range(nv):
        video_name = video_names[ii]
        run_str +=  ( ' -i ' + video_name+end_str)
    run_str += ' -filter_complex " '
    for ii in range(nv):
#         run_str += '[' +str(ii) +':v] setpts=' + str(retiming[ii]) +'*PTS, scale=1024x768 [a'+str(ii)+']; '
        run_str += '[' +str(ii) +':v] setpts=' + str(retiming[ii]) +'*PTS, scale=512x384 [a'+str(ii)+']; '

    for ri in range(n_rows):
        for ci in range(n_cols):
            run_str += '[a' + str(ci+ ri*n_cols) + ']'
        run_str += 'hstack=inputs='+str(n_cols)+'[r' + str(ri) + '];'
    for ri in range(n_rows):
        run_str +='[r' + str(ri) + ']'
    run_str += 'vstack=inputs='+str(n_rows)+'[out]"'
            
    run_str += ' -map "[out]" '
    run_str += os.path.join(data_subfolder,'output_unseen'+end_str) 
    run_str = run_str + ' -y'
    print('Running tile for goal ' + str(i) )
    status = os.system(run_str)
    print('Status (0 is good):' + str(status))
    
## concatentate videos for all goals
print('Concatenating videos')
vidlist = os.path.join(data_subfolder,'vidlist.txt')
# create a list of the video names for ffmpeg
with open(vidlist, 'w') as f:
    for i in range(6):
        end_str = str(i) + '.mp4'
        vidpath = 'output_unseen'+end_str # ffmpeg seems to pick up the paths from the location of the file
        f.write("file '" +vidpath +"'\n")
# run concat command in ffmpeg
run_str2 = 'ffmpeg -f concat -safe 0 -i ' + vidlist + ' -c copy ' + os.path.join(data_subfolder,'output.mp4')
run_str2 = run_str2 + ' -y'
status = os.system(run_str2)
print('Status (0 is good):' + str(status))
print('Done')


getting video lengths
relative retiming: [1.         0.42222222 0.8        0.45925926 0.45925926 0.59259259
 0.67407407 0.61481481 0.88888889 0.76296296 0.63703704 0.71851852]
Running tile for goal 0
Status (0 is good):0
getting video lengths
relative retiming: [0.75806452 0.56451613 0.71774194 0.45967742 0.60483871 0.62903226
 0.81451613 0.83064516 0.7016129  1.         0.76612903 0.74193548]
Running tile for goal 1
Status (0 is good):0
getting video lengths
relative retiming: [0.7578125 0.5546875 0.640625  0.7109375 0.515625  0.6328125 0.6328125
 0.6796875 0.5546875 1.        0.546875  0.6953125]
Running tile for goal 2
Status (0 is good):0
getting video lengths
relative retiming: [0.46428571 0.45714286 0.82857143 0.52857143 0.42142857 0.71428571
 0.62142857 0.67142857 1.         0.68571429 0.47142857 0.74285714]
Running tile for goal 3
Status (0 is good):0
getting video lengths
relative retiming: [0.58914729 0.7751938  0.66666667 0.48837209 0.68992248 0.69767442
 0.65891473 0.868217

In [7]:
# # # # video tile compile. This works ok
# urdf_vids = ['llllll_apply_policy.mp4',
#              'wnwwnw_apply_policy.mp4',
#              'llwwll_apply_policy.mp4',
#              'lnwwnl_apply_policy.mp4']
# vid_frames = []
# import subprocess
# for urdf_vid in urdf_vids:
#     batcmd = ('ffprobe -select_streams v -show_streams '
#            + os.path.join(data_subfolder,urdf_vid))
#     result = str(subprocess.check_output(batcmd, shell=True))
#     nb_frames_index = result.find('nb_frames=')
#     next_backslash = result[nb_frames_index:].find('\\n')
#     n_frames = result[nb_frames_index+len('nb_frames='):
#                       nb_frames_index+next_backslash]
#     vid_frames.append(int(n_frames))

# # calculate retiming
# print(np.array(vid_frames)/np.max(vid_frames))
# retiming = 2*np.max(vid_frames)/np.array(vid_frames)

# run_str = ('ffmpeg'+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[0])+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[1])+ 
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[2])+
#            ' -i '+ os.path.join(data_subfolder, urdf_vids[3])+ 
#            ' -filter_complex '+ 
#            '"nullsrc=size=2048x1536 [base]; '+
#            '[0:v] setpts=' + str(retiming[0]) +'*PTS, scale=1024x768 [upperleft]; '+
#            '[1:v] setpts=' + str(retiming[1]) +'*PTS, scale=1024x768 [upperright]; '+
#            '[2:v] setpts=' + str(retiming[2]) +'*PTS, scale=1024x768 [lowerleft]; '+
#            '[3:v] setpts=' + str(retiming[3]) +'*PTS, scale=1024x768 [lowerright]; '+
#            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
#            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
#            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
#            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
#            os.path.join(data_subfolder,'output.mp4')
#           )

# # run_str = ('ffmpeg'+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[0])+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[1])+ 
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[2])+
# #            ' -i '+ os.path.join(data_subfolder, urdf_vids[3])+ 
# #            ' -filter_complex '+ 
# #            '"nullsrc=size=2048x1536 [base]; '+
# #            '[0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; '+
# #            '[1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; '+
# #            '[2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; '+
# #            '[3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; '+
# #            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
# #            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
# #            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
# #            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768" ' +
# #            os.path.join(data_subfolder,'output.mp4')
# #           )
# status = os.system(run_str)
# print(status)
# if status != 0:
#     run_str = run_str + ' -y'
#     status = os.system(run_str)
#     print(status)



# # ffmpeg -i test.mp4 -i test.mp4 -i test2.mp4 -i test2.mp4 -filter_complex 
# # "nullsrc=size=2048x1536 [base]; [0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; [1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; [2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; [3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; [base][upperleft] overlay=shortest=1 [tmp1]; [tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; [tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; [tmp3][lowerright] overlay=shortest=1:x=1024:y=768" output.mp4

In [8]:

# import subprocess
# # run_str = ['ffmpeg',
# #            '-i', os.path.join(data_subfolder, urdf_vids[0]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[1]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[2]), 
# #            '-i', os.path.join(data_subfolder, urdf_vids[3]), 
# #            '-filter_complex', 
# #            ('"nullsrc=size=2048x1536 [base]; '+
# #            '[0:v] setpts=PTS-STARTPTS, scale=1024x768 [upperleft]; '+
# #            '[1:v] setpts=PTS-STARTPTS, scale=1024x768 [upperright]; '+
# #            '[2:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerleft]; '+
# #            '[3:v] setpts=PTS-STARTPTS, scale=1024x768 [lowerright]; '+
# #            '[base][upperleft] overlay=shortest=1 [tmp1]; '+
# #            '[tmp1][upperright] overlay=shortest=1:x=1024 [tmp2]; '+
# #            '[tmp2][lowerleft] overlay=shortest=1:y=768 [tmp3]; '+
# #            '[tmp3][lowerright] overlay=shortest=1:x=1024:y=768"'),
# #            'output.mp4'
# #             ]
# # cmd_status = subprocess.call(run_str)
# cmd_status = subprocess.call(['ffmpeg', '-i',
#                   os.path.join(data_subfolder,'output.mp4'), '-filter:v',
#                   'setpts=2.0*PTS',
#                   os.path.join(data_subfolder,'output_1x.mp4'), '-y']
# # print(cmd_status)
